In [1]:
!pip install --upgrade pyspark pandas numpy pyarrow duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.3.2
    Uninstalling duckdb-0.3.2:
      Successfully uninstalled duckdb-0.3.2


In [14]:
import os
from pyspark.sql import SparkSession
import duckdb

We'll upload the 3 data sources to a single database in 3 different tables to create a formated zone.

In [15]:
spark = SparkSession.builder.master("local[*]").appName("ParquetToDuckDB").getOrCreate()

In [17]:
for source in ["flights","airbnb","weather"]:
    df = spark.read.parquet(f"{source}_data.parquet")
    pandas_df = df.toPandas()

    con = duckdb.connect(database='/database.duckdb')

    con.register(f"{source}_view", pandas_df)

    con.execute(f"CREATE TABLE IF NOT EXISTS {source} AS SELECT * FROM {source}_view")

    result = con.execute(f"SELECT * FROM {source} LIMIT 10").fetchall()
    print(result)

[('EHAM', 4.763889, 52.308609, 'Amsterdam', ['{flight={owner={name=Austrian Airlines, logo=https://images.flightradar24.com/assets/airlines/logotypes/50.png, code={iata=OS, icao=AUA}}, identification={callsign=AUA373, number={alternative=null, default=OS373}, row=5483789749, id=3473cc55, codeshare=[ET1216]}, aircraft={country={alpha2=AT, name=Austria, alpha3=AUT, id=15}, restricted=false, hex=440D71, registration=OE-LZP, model={code=A20N, text=Airbus A320-271N}, availability={age=true, serialNo=true}, age={availability=true}, serialNo=null}, time={estimated={arrival=null, departure=null}, other={duration=5520, eta=1711034744}, real={arrival=1711034744, departure=1711029233}, scheduled={arrival=1711035900, departure=1711029000}}, airline={name=Austrian Airlines, short=Austrian Airlines, code={iata=OS, icao=AUA}}, airport={destination={timezone={name=Europe/Amsterdam, offset=3600, abbr=CET, isDst=false, abbrName=Central European Time}, info={gate=null, terminal=1, baggage=null}}, real=nu

In [18]:
con = duckdb.connect(database='/database.duckdb')

from google.colab import files
files.download('/database.duckdb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>